<a href="https://www.kaggle.com/code/kenny3s/iht-rehalf?scriptVersionId=165287913" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import os
import shutil
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio
from skimage import img_as_ubyte, io
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data_dir = '/kaggle/input/celeb-m/Celebrity Faces Dataset'

In [3]:
def block_no_expand(name):
    im=np.array(Image.open(name))
    im_lab=cv2.cvtColor(im,cv2.COLOR_RGB2HSV)
    im_lab[:,:,-1]=np.interp(im_lab[:,:,-1],(0,255),(0,255/4))
    im_rgb=cv2.cvtColor(im_lab,cv2.COLOR_HSV2RGB)
    im_ht=Image.fromarray(im_rgb).convert("1")
    im_ht=img_as_ubyte(im_ht)
    # gray_level(im_ht,(2,4),True)
    return im_ht

def find_best_params(gt_path):
    img = block_no_expand(gt_path)
    pil_halftone = Image.fromarray(img).convert('L').convert("1")
    halftone = img_as_ubyte(pil_halftone)
    ref = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)

    def IHT(radius=3, sigma_s=100, cnts=3):
        dehalftoned = halftone.copy()
        for i in range(cnts):
            dehalftoned = cv2.bilateralFilter(dehalftoned, radius, sigma_s, sigma_s)
        return dehalftoned

    best_psnr = 0
    best_params = None

    for radius in range(3, 13, 2):
        for sigma_s in range(50,400,10):
            for cnts in range(3, 10, 2):
                I = cv2.GaussianBlur(halftone.copy(), (radius, radius), 0)
                dehalftoned = IHT(radius, sigma_s, cnts)
                ehist = cv2.equalizeHist(dehalftoned)
                psnr = peak_signal_noise_ratio(ehist, ref)
                #print(f"{cnts}_{radius}_{sigma_s}_PSNR:{psnr:.4f}")
                if psnr > best_psnr:
                    best_psnr = psnr
                    best_params = (radius, sigma_s, cnts)
    return best_params, best_psnr

In [4]:
with open('best_params.txt', 'w') as f:
    f.write('id, cnts, radius, sigma_s\n')
    for img in tqdm(os.listdir(data_dir)):
        best_params, best_psnr= find_best_params(f'{data_dir}/{img}')
        print(f"best_params:{best_params}---best_psnr:{best_psnr}")
        f.write(f'{img[:-4]}, {best_params[0]}, {best_params[1]}, {best_params[2]}\n')

  0%|          | 1/1800 [00:16<8:08:25, 16.29s/it]

best_params:(5, 250, 5)---best_psnr:23.482077082290562


  0%|          | 2/1800 [00:32<8:04:44, 16.18s/it]

best_params:(5, 90, 9)---best_psnr:15.07360234201061


  0%|          | 3/1800 [00:48<8:04:59, 16.19s/it]

...

best_params:(3, 350, 9)---best_psnr:16.198803226328028


100%|█████████▉| 1799/1800 [8:08:29<00:16, 16.31s/it]

best_params:(5, 90, 9)---best_psnr:17.27824427046125


100%|██████████| 1800/1800 [8:08:45<00:00, 16.29s/it]

best_params:(5, 330, 5)---best_psnr:14.948425127250227
